In [6]:
import hashlib
from dataclasses import dataclass

# -------------------------
# Utilidades de serialización y hashing
# -------------------------
def u32_le(n: int) -> bytes:
    """Entero de 32 bits a little-endian (4 bytes)."""
    return n.to_bytes(4, byteorder="little", signed=False)

def hex_le(s: str) -> bytes:
    """
    Toma un hex mostrado en "formato humano" (big-endian),
    y lo convierte a bytes little-endian como exige el header.
    """
    b = bytes.fromhex(s)
    return b[::-1]  # invertir a little-endian

def dbl_sha256(b: bytes) -> bytes:
    """double-SHA256"""
    return hashlib.sha256(hashlib.sha256(b).digest()).digest()

# -------------------------
# Conversión nBits (compact) -> target entero
# -------------------------
def bits_to_target(bits: int) -> int:
    """
    nBits es un formato compacto: 1 byte de exponente y 3 de mantisa.
    target = mantissa * 256^(exponent-3)
    """
    exponent = bits >> 24
    mantissa = bits & 0xFFFFFF
    return mantissa * (1 << (8 * (exponent - 3)))

def hash_to_int_be(h: bytes) -> int:
    """Interpreta el hash (bytes) como entero big-endian."""
    return int.from_bytes(h, byteorder="big")

# -------------------------
# Estructura del header de bloque
# -------------------------
@dataclass
class BlockHeader:
    version: int          # 4 bytes LE
    prev_block: str       # 32 bytes (hex big-endian de cara al usuario)
    merkle_root: str      # 32 bytes (hex big-endian de cara al usuario)
    timestamp: int        # 4 bytes LE (UNIX epoch)
    bits: int             # 4 bytes LE (compact)
    nonce: int            # 4 bytes LE

    def serialize(self) -> bytes:
        """
        Serializa el header exactamente como lo hace Bitcoin:
        - version, time, bits, nonce en little-endian
        - prev_block y merkle_root como hashes little-endian (hex invertido)
        """
        return (
            u32_le(self.version) +
            hex_le(self.prev_block) +
            hex_le(self.merkle_root) +
            u32_le(self.timestamp) +
            u32_le(self.bits) +
            u32_le(self.nonce)
        )

    def pow_hash(self) -> bytes:
        """double-SHA256 del header serializado."""
        return dbl_sha256(self.serialize())

    def is_valid_pow(self) -> bool:
        target = bits_to_target(self.bits)
        h = self.pow_hash()
        return hash_to_int_be(h) <= target

In [7]:
# -------------------------
# 1) Verificar PoW del bloque génesis
#    Datos públicos conocidos
# -------------------------
genesis = BlockHeader(
    version=1,
    prev_block="00"*32,  # 32 bytes de cero
    merkle_root="4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b",
    timestamp=1231006505,   # 2009-01-03 18:15:05 UTC
    bits=0x1d00ffff,        # dificultad 1 (target inicial)
    nonce=2083236893
)

g_hash = genesis.pow_hash()               # bytes (big-endian al imprimir .hex())
g_hash_hex_be = g_hash.hex()              # representación humana
g_hash_hex_display = g_hash[::-1].hex()   # cómo se muestra típicamente en exploradores (LE->BE)

print("=== GENESIS BLOCK ===")
print("Header hash (big-endian):   ", g_hash_hex_be)
print("Hash mostrado (convención): ", g_hash_hex_display)
print("Target (desde nBits):       ", hex(bits_to_target(genesis.bits)))
print("¿PoW válido?:               ", genesis.is_valid_pow())

=== GENESIS BLOCK ===
Header hash (big-endian):    6fe28c0ab6f1b372c1a6a246ae63f74f931e8365e15a089c68d6190000000000
Hash mostrado (convención):  000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f
Target (desde nBits):        0xffff0000000000000000000000000000000000000000000000000000
¿PoW válido?:                False


In [8]:
# -------------------------
# 2) Mini “minado” educativo
#    Buscamos un nonce para un target MUY fácil (no real de Bitcoin),
#    para que encuentre solución rápido con pocos intentos.
# -------------------------
easy_header = BlockHeader(
    version=1,
    prev_block="11"*32,
    merkle_root="22"*32,
    timestamp=1700000000,
    bits=0x207fffff,  # target extremadamente fácil (exponente 0x20, mantisa ~0x7fffff)
    nonce=0
)

easy_target = bits_to_target(easy_header.bits)
print("\n=== MINERÍA EDUCATIVA (target fácil) ===")
print("Target fácil: ", hex(easy_target))

found = None
for nonce in range(0, 5_000_000):
    easy_header.nonce = nonce
    h = easy_header.pow_hash()
    if hash_to_int_be(h) <= easy_target:
        found = (nonce, h)
        break

if found:
    nonce, h = found
    print(f"¡Encontrado! nonce={nonce}")
    print("Hash:", h.hex())
    print("Hash (estilo explorador):", h[::-1].hex())
else:
    print("No se encontró un nonce en el rango probado (sube el rango o facilita más el target).")


=== MINERÍA EDUCATIVA (target fácil) ===
Target fácil:  0x7fffff0000000000000000000000000000000000000000000000000000000000
¡Encontrado! nonce=0
Hash: 74f2a921eda4c3952a6284433a184e4d8d67e0a77645cce95a72db68d0390969
Hash (estilo explorador): 690939d068db725ae9cc4576a7e0678d4d4e183a4384622a95c3a4ed21a9f274
